<a href="https://colab.research.google.com/github/DDan991/12/blob/main/CNN_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.15.0


In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#The pixel values in the images are integers between 0 and 255. By dividing each pixel by 255,
#we scale these values to a range of 0 to 1.
#Normalization is a common preprocessing step to help with the convergence of the training process.
x_train, x_test = x_train / 255, x_test / 255

#Why Do We Need a Channel Dimension?
#TensorFlow's convolutional layers expect inputs to have 4 dimensions: [batch_size, height, width, channels].
#Even if there's only one channel (as with grayscale images), you need to explicitly include this channel dimension.

#for example: "...After this operation, the shape becomes (5, 28, 28, 1). Now, each image has an explicit channel dimension of 1."

x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [17]:
# 创建 dataset，shuffle 是load 10000 elements，然后从中随机挑选
# batch(32) 是指 每个batch有32个elements

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [18]:
class Mymodel(Model):
  def __init__(self):
    super(Mymodel, self).__init__()
    self.conv1 = Conv2D(32 , 3, activation = 'relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation = 'relu')
    self.d2 = Dense(10)

  def call(self,x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = Mymodel()

In [19]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
# the loss function that the outputs of the model are logits.
# Logtis are raw predictions by a neural network, which are not normalized
# if the model inclues a softmax activation function, you should set from_logts = False

optimizer = tf.keras.optimizers.Adam()
# Adam is an adaptive learning rate optimization algorithm

In [20]:
#a 'Mean' metric object to compute the average loss over all the batches of training data

train_loss = tf.keras.metrics.Mean(name = 'train_loss')

# It calculates how often predictions match integer labels

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseTopKCategoricalAccuracy(name = 'test_accracy')

In [24]:
# complie the function into a callable TensorFlow graph

@tf.function
def train_step(images, labels):

  #the context manageer records the operations for automatic differntiation. for the backpropagation algorithm used during training
  with tf.GradientTape() as tape:

    # training is True , 当layers 是 Dropout 或者 BatchNormalization, which behave differently during the training and inference
    predictions = model(images, training = True)
    loss = loss_object(labels, predictions)

  #得到 gradients
  gradients = tape.gradient(loss, model.trainable_variables)

  #将 gradients 放入 optimizer
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


In [22]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training = False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [26]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.04472764953970909, Accuracy: 98.64666748046875, Test Loss: 0.06117803603410721, Test Accuracy: 99.98999786376953
Epoch 2, Loss: 0.02415388822555542, Accuracy: 99.20500183105469, Test Loss: 0.05286785960197449, Test Accuracy: 99.9800033569336
Epoch 3, Loss: 0.013892518356442451, Accuracy: 99.55166625976562, Test Loss: 0.05780019611120224, Test Accuracy: 99.95999908447266
Epoch 4, Loss: 0.011253850534558296, Accuracy: 99.63166809082031, Test Loss: 0.060394834727048874, Test Accuracy: 99.9800033569336
Epoch 5, Loss: 0.006118257064372301, Accuracy: 99.80333709716797, Test Loss: 0.07487668842077255, Test Accuracy: 99.97000122070312
